# RIT IMGS 211: Probability and Statistics for Imaging Scientists
### Created by Gabriel J. Diaz

![](https://www.rit.edu/brandportal/sites/rit.edu.brandportal/files/inline-images/new_RIT_full_RGB_hor_k_0.png?export=view&id=XXX)


In [1]:
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
import pandas as pd

Create a series

In [2]:
series1 = pd.Series([1,2,3])
print(series1)

0    1
1    2
2    3
dtype: int64


In [3]:
series1.index

RangeIndex(start=0, stop=3, step=1)

In [4]:
series1.values

array([1, 2, 3])

In [5]:
series3 = pd.Series([1,2,3],index=[2,3,4])
series3

2    1
3    2
4    3
dtype: int64

In [6]:
series3.iloc[2]

3

In [7]:
series2 = pd.Series([1,2,3],index=['nObservers','nAlgorithms', 'nScenes'])
series2

nObservers     1
nAlgorithms    2
nScenes        3
dtype: int64

In [8]:
series2['nAlgorithms']

2

In [9]:
ISO = pd.Series([100,200,400,800])
ISO

0    100
1    200
2    400
3    800
dtype: int64

In [10]:
ISO.min()

100

In [11]:
ISO.std()

309.56959368344513

In [12]:
ISO.describe()

count      4.000000
mean     375.000000
std      309.569594
min      100.000000
25%      175.000000
50%      300.000000
75%      500.000000
max      800.000000
dtype: float64

# Dataframes

In [13]:
ISO = pd.Series([100,200,400,800],name="sensitivity")
noise = pd.Series([0.0025, .0031, .0093, .0104],name="RMS")

camera_properties_DF = ISO.to_frame().join(noise)
camera_properties_DF

,sensitivity,RMS
0,100,0.0025
1,200,0.0031
2,400,0.0093
3,800,0.0104


# Import a dataframe

In [14]:
import os
from google.colab import drive
from zmq.sugar.frame import Frame
drive.mount("/content/drive",force_remount=True)
# os.chdir('/content/drive/MyDrive/IMGS211')

Mounted at /content/drive


In [15]:
print(os.getcwd())
os.listdir()

/content


['.config', 'drive', 'sample_data']

In [16]:
dataFolder = 'first_recording_000/'
dataFrame = pd.read_csv(dataFolder + 'gaze_positions.csv')

dataFrame.head(10)

FileNotFoundError: [Errno 2] No such file or directory: 'first_recording_000/gaze_positions.csv'

In [ ]:
len(dataFrame)

Get a row of data

In [ ]:
row = dataFrame.iloc[100]
row

Notice that the gaze point 3d is spread out over several columns. To get the gaze point on a single frame requires combining that data.

In [ ]:
[row['gaze_point_3d_x'], row['gaze_point_3d_y'], row['gaze_point_3d_z']]

That's going to be a pain to deal with!  Lets write a function that simplifies things.

The function below accepts a row number

In [ ]:
def returnPos_a(varName, df_in, row_num):
    row = df_in.iloc[row_num]
    xyz = [row[f'{varName}_x'],
          row[f'{varName}_y'],
          row[f'{varName}_z']]
    return xyz

In [ ]:
returnPos_a('gaze_point_3d', dataFrame, 100)

...but, you could also write a function that accepts a row.

In [ ]:
def returnPos_b(varName, row_in):
    xyz = [row_in[f'{varName}_x'],
          row_in[f'{varName}_y'],
          row_in[f'{varName}_z']]
    return xyz

In [ ]:
row = dataFrame.iloc[100]
returnPos_b('gaze_point_3d',row)


This can be a powerful tool, becauase now you can easily iterate over rows in a dataframe.

In [ ]:
for idx, row in dataFrame.head(10).iterrows():
  print(returnPos_b('gaze_point_3d',row))

You can use a similar approach to run arbitrary functions per-row.
What if I wanted to convert the XYZ direction of gaze into the more interpretable "spherical coordinates" of azimuth and elevation?

<img src='https://www.mathworks.com/help/matlab/ref/math_sphcart.png'>

In [ ]:
def calculate_azimuth_elevation(row_in,varName):

    x,y,z = returnPos_b('gaze_point_3d',row_in)

    az = np.rad2deg(np.arctan(np.divide(x,z)))
    el = np.rad2deg(np.arctan(np.divide(y,z)))

    return (az,el)

In [ ]:
calculate_azimuth_elevation(dataFrame.iloc[100],'gaze_point_3d')

We can do this for many rows at a time, and save our results

In [ ]:
az_el = []
for idx, row in dataFrame.iterrows():
  az_el.append(calculate_azimuth_elevation(row,'gaze_point_3d'))

In [ ]:
az_el[:10]

Notice that this is a list of two-element tuples.  We could use a for loop to divide this up into a seperate list for azimuth and elevation.  An easier way is to "unzip" the list into two seperate variables.


In [ ]:
az, el = zip(*az_el[:10])
az

We can use this trick to add the data directly to our dataframe

In [ ]:
dataFrame['gazeInWorldCam_az'], dataFrame['gazeInWorldCam_el'] = zip(*az_el)
list(dataFrame)

In [ ]:
dataFrame.head(10)['gazeInWorldCam_az']

In [ ]:
dataFrame['gazeInWorldCam_az'].plot()

In [ ]:
dataFrame['gazeInWorldCam_az'][:100].plot()

Let's plot it!  I'm going to use a plotting module that I sometimes use: Plotly.  It's not great at everything, but it's great at *some* things.  I don't think it's worth your time to learn how to use it just yet.

In [ ]:
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
import os

In [ ]:
startTimeS = 42
endTimeS = 144

dataFrame['vidTimeStamp'] = np.array(dataFrame['gaze_timestamp'] - dataFrame['gaze_timestamp'].iloc[0])
plIndex = np.arange(len(dataFrame['vidTimeStamp']))

# I googled for, "python find first larger than"
startIndex = next(idx for idx, val in enumerate(dataFrame['vidTimeStamp']) if val >  startTimeS)
endIndex = next(idx for idx, val in enumerate(dataFrame['vidTimeStamp']) if val >  endTimeS)
print('Plotting from index: ' + str(startIndex) + ' to ' + str(endIndex))

yLim=[-50,50]
width=800
height=600
inline=True

# import plotly.plotly as py
import plotly.graph_objs as go
import pandas as pd

colors_idx = ['rgb(0,204,204)','rgb(128,128,128)','rgb(204,0,0)','rgb(102,0,204)']

traces = []

eih_el = go.Scattergl(
    x=dataFrame['vidTimeStamp'][startIndex:endIndex],
    y=dataFrame['gazeInWorldCam_el'][startIndex:endIndex],
    name = 'gazeInWorldCam_el',
    marker_color = colors_idx[0],
    mode='lines+markers',
    marker_size = 5,
    opacity = 0.8)

traces.append(eih_el)

eih_az = go.Scattergl(
    x=dataFrame['vidTimeStamp'][startIndex:endIndex],
    y=dataFrame['gazeInWorldCam_az'][startIndex:endIndex],
    name = 'gazeInWorldCam_az',
    marker_color = colors_idx[1],
    mode='lines+markers',
    marker_size = 5,
    opacity = 0.8)

traces.append(eih_az)


################################################################

layout = dict(
    dragmode= 'pan',
    title='Time Series with Rangeslider',
    width=width,
    height=height,
    yaxis=dict(range=yLim, title='angular position (degrees)'),
    xaxis=dict(rangeslider=dict(visible=True),range = [startTimeS,startTimeS+3])

)


fig = dict(data=traces, layout=layout)


iplot(fig)